In [3]:
from datetime import date
import calendar as c
import pandas as pd
import numpy as np

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 1500)
pd.options.display.max_colwidth = 100000

#### Para testear las funciones, levanto los dataframes originales

In [5]:
df_pageviews = pd.read_csv('data/pageviews.csv', parse_dates=['FEC_EVENT'])
df_devicedata = pd.read_csv('data/device_data.csv', parse_dates=["FEC_EVENT"])
year = 2018
mes_snapshot = 9

def universo_train(df):
    universo = pd.DataFrame()
    universo['user_id'] = df.USER_ID.unique()
    universo['snapshot_mes'] = mes_snapshot
    return universo

universo = universo_train(df_pageviews)

In [6]:
def _sum_campo_user(df_pageviews, campo):
    temp = pd.crosstab(df_pageviews.USER_ID, df_pageviews[campo]) 
    temp.columns = [campo + "_" + str(v) for v in temp.columns]
    return _df_refactor(temp)

In [7]:
def _df_refactor(df_temp):
    df_union = pd.merge(universo, df_temp, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df_union = df_union.drop(['snapshot_mes'], axis=1)
    df_union = df_union.sort_values('user_id')
    return df_union

In [8]:
def _sum_campo_user_between(df_pageviews, fecha_desde, fecha_hasta, campo):
    df = df_pageviews[fecha_desde <= df_pageviews['FEC_EVENT']]
    df = df[df['FEC_EVENT'] <= fecha_hasta]
    return _sum_campo_user(df, campo)

In [9]:
def _sum_total_user(df_pageviews, campo):
    temp = df_pageviews.groupby("USER_ID")[campo].sum().to_frame().reset_index()
    temp = _df_refactor(temp)
    temp = temp.drop(['USER_ID'], axis=1)
    return temp

In [10]:
def _sum_total_user_between(df_pageviews, fecha_desde, fecha_hasta, campo):
    df = df_pageviews[fecha_desde <= df_pageviews['FEC_EVENT']]
    df = df[df['FEC_EVENT'] <= fecha_hasta]
    return _sum_total_user(df, campo)

In [11]:
def _df_complete_columns(df_complete_original, df_incomplete):
    df_complete = df_complete_original
    df_complete[:] = 0
    df = pd.DataFrame()
    for a_column in df_complete:
        if a_column in df_incomplete.columns:
            df[a_column] = df_incomplete[a_column]
        else:
            df[a_column] = df_complete[a_column]
    return df

In [12]:
def _df_repeat_columns(df_complete_original, df_incomplete):
    df_complete = df_complete_original
    df_complete[:] = df_incomplete
    return df_complete

In [13]:
def _sum_user_campo_all(df_pageviews, mes_snapshot, campo):
    fecha_desde_3m = pd.Timestamp(year, mes_snapshot - 2, 1)
    fecha_desde_1m = pd.Timestamp(year, mes_snapshot, 1)
    fecha_desde_15d = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1] - 15)
    fecha_desde_7d = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1] - 7)
    fecha_desde_1d = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1] - 1)
    fecha_hasta_last_day = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1])
    
    df_3m = _sum_campo_user_between(df_pageviews, fecha_desde_3m, fecha_hasta_last_day, campo)
    df_1m = _sum_campo_user_between(df_pageviews, fecha_desde_1m, fecha_hasta_last_day, campo)
    df_15d = _sum_campo_user_between(df_pageviews, fecha_desde_15d, fecha_hasta_last_day, campo)
    df_7d = _sum_campo_user_between(df_pageviews, fecha_desde_7d, fecha_hasta_last_day, campo)
    df_1d =_sum_campo_user_between(df_pageviews, fecha_desde_1d, fecha_hasta_last_day, campo)
    df_ever =_sum_campo_user(df_pageviews, campo)

    tot_3m = _sum_total_user_between(df_pageviews, fecha_desde_3m, fecha_hasta_last_day, campo)
    tot_1m = _sum_total_user_between(df_pageviews, fecha_desde_1m, fecha_hasta_last_day, campo)
    tot_15d = _sum_total_user_between(df_pageviews, fecha_desde_15d, fecha_hasta_last_day, campo)
    tot_7d = _sum_total_user_between(df_pageviews, fecha_desde_7d, fecha_hasta_last_day, campo)    
    tot_1d = _sum_total_user_between(df_pageviews, fecha_desde_1d, fecha_hasta_last_day, campo)
    tot_ever = _sum_total_user(df_pageviews, campo)

    tot_3m = _df_repeat_columns(df_ever, tot_3m)
    tot_1m = _df_repeat_columns(df_ever, tot_1m)
    tot_15d = _df_repeat_columns(df_ever, tot_15d)
    tot_7d = _df_repeat_columns(df_ever, tot_7d)
    tot_1d = _df_repeat_columns(df_ever, tot_1d)
    tot_ever = _df_repeat_columns(df_ever, tot_ever)
    
    df_3m =_df_complete_columns(df_ever, df_3m)
    df_1m =_df_complete_columns(df_ever, df_1m)
    df_15d =_df_complete_columns(df_ever, df_15d)
    df_7d =_df_complete_columns(df_ever, df_7d)
    df_1d =_df_complete_columns(df_ever, df_1d)
    
    return df_3m, df_1m, df_15d, df_7d, df_1d, df_ever, tot_3m, tot_1m, tot_15d, tot_7d, tot_1d, tot_ever

In [14]:
def _ratio_campo_df(df_numerador, df_denominador, name):
    #Eliminar columnar user_id para que no la divida
    users = df_numerador['user_id']
    df_numerador = df_numerador.drop(['user_id'], axis = 1)
    df_denominador = df_denominador.drop(['user_id'], axis = 1)
    
    #División element-wise
    temp = df_numerador.div(df_denominador, axis=0, fill_value = 0)
    temp.columns = [name + "_" + str(v) for v in temp.columns]

    #Agrega columna user_id
    temp['user_id'] = users

    #Completa Nan
    temp = temp.fillna(-999)
    return temp

In [15]:
def _ratio_campo_df_series(df_numerador, serie_denominador, name):
    #Eliminar columnar user_id para que no la divida
    users = df_numerador['user_id']
    df_numerador = df_numerador.drop(['user_id'], axis = 1)

    #División element-wise
    temp = df_numerador.div(serie_denominador, axis=0)
    temp.columns = [name + "_" + str(v) for v in temp.columns]

    #Agrega columna user_id
    temp['user_id'] = users

    #Completa Nan
    temp = temp.fillna(-999)
    return temp

In [16]:
def _ratio_campo_series(serie_numerador, serie_denominador, name):
    #División element-wise
    temp = df_numerador.div(serie_denominador, axis=0)
    temp = temp.to_frame().reset_index()
    
    temp.columns = ['user_id', 'name_' + temp.columns[1]]
    
    #Completa Nan
    temp = temp.fillna(-999)
    return temp

In [52]:
def _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo):
    df_3m, df_1m, df_15d, df_7d, df_1d, df_ever, tot_3m, tot_1m, tot_15d, tot_7d, tot_1d, tot_ever = _sum_user_campo_all(df_pageviews, mes_snapshot, campo)
    print('llegue 1')
    ratio_preferencia_3m = _ratio_campo_df(df_3m, tot_3m, 'ratio_preferencia_3m')
    ratio_preferencia_1m = _ratio_campo_df(df_1m, tot_1m, 'ratio_preferencia_1m')
    ratio_preferencia_15d = _ratio_campo_df(df_15d, tot_15d, 'ratio_preferencia_15d')
    ratio_preferencia_7d = _ratio_campo_df(df_7d, tot_7d, 'ratio_preferencia_7d')
    ratio_preferencia_1d = _ratio_campo_df(df_1d, tot_1d, 'ratio_preferencia_1d')
    print('llegue 2')
    ratio_interes_3m = _ratio_campo_df(df_3m, df_ever, 'ratio_interes_3m')
    ratio_interes_1m = _ratio_campo_df(df_1m, df_ever, 'ratio_interes_1m')
    ratio_interes_15d = _ratio_campo_df(df_15d, df_ever, 'ratio_interes_15d')
    ratio_interes_7d = _ratio_campo_df(df_7d, df_ever, 'ratio_interes_7d')
    ratio_interes_1d = _ratio_campo_df(df_1d, df_ever, 'ratio_interes_1d')
    print('llegue 3')
    ratio_interes_reciente_1m = _ratio_campo_df(df_1m, df_3m, 'ratio_interes_reciente_1m')
    ratio_interes_reciente_15d = _ratio_campo_df(df_15d, df_3m, 'ratio_interes_reciente_15d')
    ratio_interes_reciente_7d = _ratio_campo_df(df_7d, df_3m, 'ratio_interes_reciente_7d')
    ratio_interes_reciente_1d = _ratio_campo_df(df_1d, df_3m, 'ratio_interes_reciente_1d')
    print('llegue 4')
    ratio_actividad_3m = _ratio_campo_df(tot_3m.iloc[:,0:2], tot_ever.iloc[:,0:2], 'ratio_actividad_3m')
    ratio_actividad_1m = _ratio_campo_df(tot_1m.iloc[:,0:2], tot_ever.iloc[:,0:2], 'ratio_actividad_1m')
    ratio_actividad_15d = _ratio_campo_df(tot_15d.iloc[:,0:2], tot_3m.iloc[:,0:2], 'ratio_actividad_15d')
    ratio_actividad_7d = _ratio_campo_df(tot_7d.iloc[:,0:2], tot_3m.iloc[:,0:2], 'ratio_actividad_7d')
    ratio_actividad_1d = _ratio_campo_df(tot_1d.iloc[:,0:2], tot_3m.iloc[:,0:2], 'ratio_actividad_1d')
    print('llegue 5')
    df_union = pd.merge(universo, ratio_preferencia_3m, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_preferencia_1m, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_preferencia_15d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_preferencia_7d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_preferencia_1d, how='left', left_on=['user_id'], right_on=['user_id'])
    print('llegue 6')
    df_union = pd.merge(df_union, ratio_interes_3m, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_interes_1m, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_interes_15d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_interes_7d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_interes_1d, how='left', left_on=['user_id'], right_on=['user_id'])
    print('llegue 7')
    df_union = pd.merge(df_union, ratio_interes_reciente_1m, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_interes_reciente_15d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_interes_reciente_7d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_interes_reciente_1d, how='left', left_on=['user_id'], right_on=['user_id'])
    print('llegue 8')
    df_union = pd.merge(df_union, ratio_actividad_3m, how='left', left_on=['user_id'], right_on=['user_id'] )
    df_union = pd.merge(df_union, ratio_actividad_1m, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_actividad_15d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_actividad_7d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_actividad_1d, how='left', left_on=['user_id'], right_on=['user_id'])
    print('llegue 9')
    return df_union

In [ ]:
#PROBAR
temp = ratio_page_all(universo, df_pageviews, mes_snapshot)

llegue 1
llegue 2
llegue 3
llegue 4
llegue 5
llegue 6
llegue 7
llegue 8


In [ ]:
temp.head(2)

In [50]:
ratio_actividad_1d = _ratio_campo_df(tot_1d.iloc[:,0:2], tot_3m.iloc[:,0:2], 'ratio_actividad_1d')

In [18]:
df_3m, df_1m, df_15d, df_7d, df_1d, df_ever, tot_3m, tot_1m, tot_15d, tot_7d, tot_1d, tot_ever = _sum_user_campo_all(df_pageviews, mes_snapshot, 'PAGE')
    

In [51]:
ratio_actividad_1d.head(10)

,ratio_actividad_1d_PAGE_1,user_id
0,-999.0,0
1,-999.0,0
2,-999.0,0
3,-999.0,0
4,-999.0,0
5,-999.0,0
6,-999.0,0
7,-999.0,0
8,-999.0,0
9,-999.0,0


In [48]:
tot_1m.iloc[:,0:2]

,user_id,PAGE_1
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


In [ ]:
    ratio_actividad_3m = _ratio_campo_df(tot_3m[1], tot_ever, 'ratio_actividad_3m')

In [49]:
tot_1m.head(10)

user_id  PAGE_1  PAGE_2  PAGE_3  PAGE_4  PAGE_5  PAGE_6  PAGE_7  PAGE_8  \
0        0       0       0       0       0       0       0       0       0   
1        0       0       0       0       0       0       0       0       0   
2        0       0       0       0       0       0       0       0       0   
3        0       0       0       0       0       0       0       0       0   
4        0       0       0       0       0       0       0       0       0   
5        0       0       0       0       0       0       0       0       0   
6        0       0       0       0       0       0       0       0       0   
7        0       0       0       0       0       0       0       0       0   
8        0       0       0       0       0       0       0       0       0   
9        0       0       0       0       0       0       0       0       0   

   PAGE_9  PAGE_10  PAGE_11  PAGE_12  PAGE_13  PAGE_14  PAGE_15  PAGE_16  \
0       0        0        0        0        0        0        0        0   
1       0        0        0        0        0        0        0        0   
2       0        0        0        0        0        0        0        0   
3       0        0        0        0        0        0        0        0   
4       0        0        0        0        0        0        0        0   
5       0        0        0        0        0        0        0        0   
6       0        0        0        0        0        0        0        0   
7       0        0        0        0        0        0        0        0   
8       0        0        0        0        0        0        0        0   
9       0        0        0        0        0        0        0        0   

   PAGE_17  PAGE_18  PAGE_19  PAGE_20  PAGE_21  PAGE_22  PAGE_23  PAGE_24  \
0        0        0        0        0        0        0        0        0   
1        0        0        0        0        0        0        0        0   
2        0        0        0        0        0        0        0        0   
3        0        0        0        0        0        0        0        0   
4        0        0        0        0        0        0        0        0   
5        0        0        0        0        0        0        0        0   
6        0        0        0        0        0        0        0        0   
7        0        0        0        0        0        0        0        0   
8        0        0        0        0        0        0        0        0   
9        0        0        0        0        0        0        0        0   

   PAGE_25  PAGE_26  PAGE_27  PAGE_28  PAGE_29  PAGE_30  PAGE_31  PAGE_32  \
0        0        0        0        0        0        0        0        0   
1        0        0        0        0        0        0        0        0   
2        0        0        0        0        0        0        0        0   
3        0        0        0        0        0        0        0        0   
4        0        0        0        0        0        0        0        0   
5        0        0        0        0        0        0        0        0   
6        0        0        0        0        0        0        0        0   
7        0        0        0        0        0        0        0        0   
8        0        0        0        0        0        0        0        0   
9        0        0        0        0        0        0        0        0   

   PAGE_33  PAGE_34  PAGE_35  PAGE_36  PAGE_37  PAGE_38  PAGE_39  PAGE_40  \
0        0        0        0        0        0        0        0        0   
1        0        0        0        0        0        0        0        0   
2        0        0        0        0        0        0        0        0   
3        0        0        0        0        0        0        0        0   
4        0        0        0        0        0        0        0        0   
5        0        0        0        0        0        0        0        0   
6        0        0        0        0        0        0        0        

In [54]:
def ratio_page_all(universo, df_pageviews, mes_snapshot):
    campo = 'PAGE'
    return _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo)

In [ ]:
def ratio_ccategory_all(universo, df_pageviews, mes_snapshot):
    campo = 'CONTENT_CATEGORY'
    return _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo)

In [ ]:
def ratio_cctop_all(universo, df_pageviews, mes_snapshot):
    campo = 'CONTENT_CATEGORY_TOP'
    return _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo)

In [ ]:
def ratio_ccbottom_all(universo, df_pageviews, mes_snapshot):
    campo = 'CONTENT_CATEGORY_BOTTOM'
    return _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo)

In [ ]:
def ratio_site_id_all(universo, df_pageviews, mes_snapshot):
    campo = 'SITE_ID'
    return _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo)

In [ ]:
def ratio_mobile_device_all(universo, df_devicedata, mes_snapshot):
    campo = 'IS_MOBILE_DEVICE'
    return _ratio_campo_all(universo, df_devicedata, mes_snapshot, campo)

In [ ]:
def ratio_connection_speed_all(universo, df_devicedata, mes_snapshot):
    campo = 'CONNECTION_SPEED'
    return _ratio_campo_all(universo, df_devicedata, mes_snapshot, campo)

In [ ]:
def ratio_on_site_search_term_all(universo, df_devicedata, mes_snapshot):
    campo = 'ON_SITE_SEARCH_TERM'
    return _ratio_campo_all(universo, df_devicedata, mes_snapshot, campo)